<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [108]:
df = pd.read_csv("content/2024.csv")

In [109]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  128 non-null    object
 1   AP        128 non-null    object
 2   N01       128 non-null    int64 
 3   N02       128 non-null    int64 
 4   N03       128 non-null    int64 
 5   N04       128 non-null    int64 
 6   N05       128 non-null    int64 
 7   N06       128 non-null    int64 
 8   N07       128 non-null    int64 
 9   N08       128 non-null    int64 
 10  N09       128 non-null    int64 
 11  N10       128 non-null    int64 
 12  N11       128 non-null    int64 
 13  N12       128 non-null    int64 
 14  N13       128 non-null    int64 
 15  N14       128 non-null    int64 
 16  N15       128 non-null    int64 
 17  N16       128 non-null    int64 
 18  N17       128 non-null    int64 
 19  N18       128 non-null    int64 
 20  N19       128 non-null    int64 
 21  N20       128 no

In [110]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [111]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 10
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [112]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [113]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 30

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [114]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=95)

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/1000
4/4 - 14s - loss: 1.0226 - accuracy: 0.0424 - 14s/epoch - 4s/step
Epoch 2/1000
4/4 - 0s - loss: 1.0150 - accuracy: 0.0424 - 75ms/epoch - 19ms/step
Epoch 3/1000
4/4 - 0s - loss: 1.0102 - accuracy: 0.1017 - 75ms/epoch - 19ms/step
Epoch 4/1000
4/4 - 0s - loss: 1.0017 - accuracy: 0.1186 - 99ms/epoch - 25ms/step
Epoch 5/1000
4/4 - 0s - loss: 0.9969 - accuracy: 0.1186 - 99ms/epoch - 25ms/step
Epoch 6/1000
4/4 - 0s - loss: 0.9914 - accuracy: 0.1186 - 88ms/epoch - 22ms/step
Epoch 7/1000
4/4 - 0s - loss: 0.9815 - accuracy: 0.1017 - 87ms/epoch - 22ms/step
Epoch 8/1000
4/4 - 0s - loss: 0.9710 - accuracy: 0.1017 - 89ms/epoch - 22ms/step
Epoch 9/1000
4/4 - 0s - loss: 0.9589 - accuracy: 0.1356 - 82ms/epoch - 20ms/step
Epoch 10/1000
4/4 - 0s - loss: 0.9461 - accuracy: 0.1186 - 80ms/epoch - 20ms/step
Epoch 11/1000
4/4 - 0s - loss: 0.9295 - accuracy: 0.1356 - 80ms/epoch - 20ms/step
Epoch 12/1000
4/4 - 0s - loss: 0.9088 - accuracy: 0.1102 - 83ms/epoch - 21ms/step
Epoch 13/1000
4/4 - 0s - lo

In [115]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

4/4 [==============================] - 3s 13ms/step - loss: 0.0027 - accuracy: 0.9153
Test Loss: 0.002697469899430871, Test Accuracy: 0.9152542352676392
4/4 [==============================] - 0s 11ms/step - loss: 0.0027 - accuracy: 0.9115
Test Loss: 0.002668430795893073, Test Accuracy: 0.9115044474601746


In [116]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [117]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-117-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
118,4,6,10,14,15,27,31,32,34,35,40,42,43,44,46,56,63,65,66,70
119,3,9,14,27,28,32,35,37,38,39,40,44,51,53,55,59,62,65,66,69
120,4,6,10,13,15,21,24,28,31,32,37,39,42,48,51,52,59,61,63,68
121,2,3,6,11,21,24,30,31,33,34,35,37,39,41,43,44,48,56,68,70
122,3,4,10,12,13,14,15,22,31,33,36,43,48,50,59,60,63,65,66,68
123,10,11,15,17,18,23,27,30,37,39,43,47,48,51,52,56,61,63,65,67
124,1,3,7,15,20,26,27,33,34,38,40,41,46,48,51,58,61,62,63,70
125,1,9,10,19,20,22,24,26,28,31,34,37,38,42,45,55,60,61,67,70
126,2,3,4,8,10,14,18,20,30,31,33,44,45,48,57,59,60,61,68,70


In [125]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(scaler.inverse_transform(y_pred).astype(int)[0])
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)
prediction = scaler.inverse_transform(y_pred).astype(int)

top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)

print("The predicted numbers for the lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("winners:", duplicates)
print(len(duplicates)/20)


1/1 [==============================] - 0s 33ms/step
The top 5 indices [19 17 16 18 14 15 13  4 12  9  5 11  6 10  7  2  3  8  1  0]
The top 5 numbers: [65, 57, 55, 62, 49, 53, 48, 19, 46, 35, 23, 43, 26, 40, 30, 13, 18, 35, 11, 7]
The predicted numbers for the lottery game are: [ 7 11 13 18 19 23 26 30 35 35 40 43 46 48 49 53 55 57 62 65]
The actual numbers in the last lottery game were: [ 7 11 13 18 19 23 26 30 35 35 40 43 46 48 49 53 55 57 62 65]


winners: [23 35 40 49 62]
0.25


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


# Predict the next set of numbers

In [128]:
next = df1.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)


1/1 [==============================] - 0s 43ms/step
The predicted numbers for the lottery game: [ 7 11 13 18 19 23 26 30 35 35 40 43 46 48 49 53 55 57 62 65]


The top 5 indices [ 7 17  4  2  5 19  3 18  6 16  8  1 15 14  0  9 13 12 10 11]
The top 5 numbers: [19, 54, 10, 4, 14, 64, 9, 59, 19, 53, 27, 5, 51, 48, 2, 34, 47, 45, 39, 43]


In [122]:
next = df1.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_pred[0], axis=0)[-10:]
top_5_indices = np.argsort(y_pred[0], axis=0)[-5:]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 23ms/step
The top 5 indices [2 3 8 1 0]
The top 5 numbers: [4, 9, 27, 5, 2]


The top 10 indices [ 5 11  6 10  7  2  3  8  1  0]
The top 10 numbers: [14, 43, 19, 39, 19, 4, 9, 27, 5, 2]


The predicted numbers: [[ 2  5  4  9 10 14 19 19 27 34 39 43 45 47 48 51 53 54 59 64]]
